## Import Library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## Load Dataset

In [ ]:
df = pd.read_csv(r"C:\Users\HP\Documents\Belajar Adit\mih-de-test\data\transaction.csv", parse_dates=["trx_date"], index_col="trx_date")
df

## Data Preprocessing

In [ ]:
df = df.sort_index()
daily_sales = df.groupby(df.index)["item_quantity"].sum()

# Normalize data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(daily_sales.values.reshape(-1, 1))

## Create Sequences

In [ ]:
def create_sequences(data, seq_length=30):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i : i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 30
X, y = create_sequences(data_scaled, seq_length)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

## Build LSTM Model

In [ ]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)
])

In [ ]:
model.compile(optimizer='adam', loss='mse')

## Model Training

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

## Make Predictions

In [ ]:
y_pred = model.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred) 
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

## Plot Results

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test_inv, label='Actual Sales', color='blue')
plt.plot(y_pred_inv, label='Predicted Sales', color='red')
plt.xlabel('Days')
plt.ylabel('Sales')
plt.legend()
plt.title('Sales Forecasting with LSTM')
plt.show()

## Predict Future Sales (30 Days)

In [ ]:
def predict_future(model, data, steps=30):
    last_sequence = data[-seq_length:]
    predictions = []
    for _ in range(steps):
        pred = model.predict(last_sequence.reshape(1, seq_length, 1))
        predictions.append(pred[0, 0])
        last_sequence = np.append(last_sequence[1:], pred).reshape(seq_length, 1)
    return scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

future_sales = predict_future(model, data_scaled, steps=30)
plt.figure(figsize=(12, 6))
plt.plot(future_sales, marker='o', linestyle='dashed', color='green', label='Future Sales Prediction')
plt.xlabel('Future Days')
plt.ylabel('Sales')
plt.legend()
plt.title('Future Sales Prediction for Next 30 Days')
plt.show()